<a href="https://colab.research.google.com/github/AlyssonBaumgarten/reconhecimento/blob/main/Tarefa_3_Algoritmo_KNN__hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from sklearn import datasets
from sklearn.decomposition import PCA

In [24]:
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target


In [25]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

In [26]:
modelo = KNeighborsRegressor()
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
print(scores)
sempad = np.mean(scores['test_score'])
print(f"Sem padronização: {sempad}")

[0.06324555 0.45607017 0.67330033 0.51510517 0.69761498]
{'fit_time': array([0.00106239, 0.00066113, 0.00062919, 0.00060296, 0.00066781]), 'score_time': array([0.00137639, 0.00097942, 0.00092125, 0.0009079 , 0.00265026]), 'test_score': array([0.06324555, 0.45607017, 0.67330033, 0.51510517, 0.69761498])}
Sem padronização: 0.48106724085424457


In [27]:
from sklearn.model_selection import GridSearchCV

parametros = {'n_neighbors': [1,3,5]}

modelo = GridSearchCV(KNeighborsRegressor(), parametros, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
modelo.fit(X, y)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [28]:
modelo.cv_results_


{'mean_fit_time': array([0.0018218 , 0.00093622, 0.00063424]),
 'std_fit_time': array([2.04150975e-03, 4.73417403e-04, 1.51146097e-05]),
 'mean_score_time': array([0.00107627, 0.00137744, 0.00090623]),
 'std_score_time': array([1.00926434e-04, 5.27165478e-04, 1.69334268e-05]),
 'param_n_neighbors': masked_array(data=[1, 3, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}],
 'split0_test_score': array([-0.        , -0.        , -0.06324555]),
 'split1_test_score': array([-0.57735027, -0.50184844, -0.45607017]),
 'split2_test_score': array([-0.85634884, -0.75767676, -0.67330033]),
 'split3_test_score': array([-0.65828059, -0.55444333, -0.51510517]),
 'split4_test_score': array([-0.81649658, -0.704483  , -0.69761498]),
 'mean_test_score': array([-0.58169526, -0.50369031, -0.48106724]),
 'std_test_score': array([0.30816224, 0.26874152, 0.22812491]),
 'rank_test_score': ar

In [29]:
modelo.best_estimator_


KNeighborsRegressor()

In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt

ypred = modelo.predict(X)
mse = mean_squared_error(y, ypred)
sqrt(mse)

0.3278210894171799

In [32]:
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False), return_estimator=True)
print(scores['test_score'])
sempad = np.mean(scores['test_score'])
print(f"Sem padronização: {sempad}")

[0.06324555 0.45607017 0.67330033 0.51510517 0.69761498]
Sem padronização: 0.48106724085424457


In [33]:
scores

{'fit_time': array([0.02595687, 0.02063894, 0.02120399, 0.02123642, 0.02047682]),
 'score_time': array([0.00060916, 0.00066352, 0.00066328, 0.00063276, 0.00065494]),
 'estimator': [GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=KNeighborsRegressor(),
               param_grid={'n_neighbors': [1, 3, 5]},
               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False)),
  GridSearchCV(estimator=

In [34]:
for estimator in scores['estimator']:
    print(estimator.best_estimator_)

KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()
KNeighborsRegressor()


In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

parametros = {'n_neighbors':  [1,3,5]}
GridSearchKNN = GridSearchCV(KNeighborsRegressor(), parametros, scoring='neg_root_mean_squared_error')

modelo = Pipeline([
    ("padronização", StandardScaler()),
    ("gsknn", GridSearchKNN)
])
scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

[0.03651484 0.49814471 0.66332496 0.52915026 0.70521864]
Com padronização: 0.48647068114050773


In [42]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

parametros = {'knn__n_neighbors': [1,3,5]}

modelo = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

scores = cross_validate(modelo, X, y, scoring=make_scorer(mean_squared_error, squared=False))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

[0.03651484 0.49814471 0.66332496 0.56764621 0.70521864]
Com padronização: 0.49416987113743344
